In [1]:
library(Seurat)
library(Signac)
library(SeuratWrappers)
library(RENIN)
library(tidyverse)
library(chromVARmotifs)
library(BSgenome.Hsapiens.UCSC.hg19)

Attaching SeuratObject

Attaching sp


Attaching package: ‘Signac’


The following object is masked from ‘package:Seurat’:

    FoldChange


── Attaching packages ──────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ─────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::reduce() masks Signac::reduce()


Warning message:
“no function found corresponding to methods exports from ‘BSgenome’ for: ‘releaseName’”
Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, cluste

## define the two ident to compare in FindMarkers function

In [2]:
FindMarkers_ident_1 = "Papilla"
FindMarkers_ident_2 = "Medulla"

celltype_subset = c("PC1", "PC2")

In [3]:
de_gene_file = "../../processed_data/RENIN/de_genes/446267_cells_RNA/PC_rank_genes_groups_p_vs_m_20230425.RData"

In [4]:
processed_dir = file.path("..", "..", "processed_data", "RENIN", "by_regions", paste0(gsub(pattern = "\\/", "_", paste(celltype_subset, collapse = "_"))))
processed_dir
dir.create(processed_dir, recursive = T, showWarnings = F)

[1] "../../processed_data/RENIN/by_regions/PC1_PC2"

### load previously calculated image which includes expr_mat, peak_mat, novaseq.sub. This could save time for dealing with new de_gene for different clusters while sharing the same novaseq.sub object

In [5]:
Sys.time()
load("../../processed_data/RENIN/RENIN_324701_cells_preprocess.RData")
Sys.time()

[1] "2023-07-03 14:52:54 CDT"

[1] "2023-07-03 14:58:53 CDT"

In [6]:
novaseq.sub
table(novaseq.sub$celltype5_rna)

An object of class Seurat 
237522 features across 324701 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: SCT
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, umap.peaks, WNN.UMAP


      JGA       POD       PEC        PT PT_dediff  PT_VCAM1       tL1       tL2 
     2465      3179      1239     64880     29494      8445     25438      9029 
   tL-TAL      TAL1      TAL2      TAL3       DCT       CNT       PC1       PC2 
    15485     29910     19724      8900     14150     13197     16140      4049 
      ICA       ICB      Uro1      Uro2      ENDO      SMC1      SMC2      SMC3 
    10579      4120      3210       682     12847      4272       156        86 
     Fib1      Fib2      Fib3        Ma        BT 
     3786      7191      3095      3673      5280 

In [7]:
ls()

[1] "celltype_subset"                  "de_gene_file"                    
 [3] "expr_mat"                         "FindMarkers_ident_1"             
 [5] "FindMarkers_ident_2"              "level.novaseq"                   
 [7] "level.novaseq.renal_region_new"   "level.novaseq.rna"               
 [9] "novaseq.sub"                      "palette.novaseq"                 
[11] "palette.novaseq.renal_region_new" "palette.novaseq.rna"             
[13] "peak_mat"                         "processed_dir"

In [8]:
## subset novaseq.sub again to only include the celltypes of interest

In [9]:
novaseq.sub = subset(novaseq.sub, celltype5_rna %in% celltype_subset)
novaseq.sub
table(novaseq.sub$celltype5_rna)

An object of class Seurat 
237522 features across 20189 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: SCT
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, umap.peaks, WNN.UMAP


      JGA       POD       PEC        PT PT_dediff  PT_VCAM1       tL1       tL2 
        0         0         0         0         0         0         0         0 
   tL-TAL      TAL1      TAL2      TAL3       DCT       CNT       PC1       PC2 
        0         0         0         0         0         0     16140      4049 
      ICA       ICB      Uro1      Uro2      ENDO      SMC1      SMC2      SMC3 
        0         0         0         0         0         0         0         0 
     Fib1      Fib2      Fib3        Ma        BT 
        0         0         0         0         0 

In [10]:
ls()

[1] "celltype_subset"                  "de_gene_file"                    
 [3] "expr_mat"                         "FindMarkers_ident_1"             
 [5] "FindMarkers_ident_2"              "level.novaseq"                   
 [7] "level.novaseq.renal_region_new"   "level.novaseq.rna"               
 [9] "novaseq.sub"                      "palette.novaseq"                 
[11] "palette.novaseq.renal_region_new" "palette.novaseq.rna"             
[13] "peak_mat"                         "processed_dir"

### remove the following variables because they will be regenerated for new comparison

In [11]:
rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", "aen_lists", "tf_results", "regulator_tf_names", "centrality_rankings"))

Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'de_genes' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'de_genes_BK' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'gene_list' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'peak_results' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'aen_lists' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'tf_results' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'regulator_tf_names' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'centrality_rankings' not found”


In [12]:
ls()

[1] "celltype_subset"                  "de_gene_file"                    
 [3] "expr_mat"                         "FindMarkers_ident_1"             
 [5] "FindMarkers_ident_2"              "level.novaseq"                   
 [7] "level.novaseq.renal_region_new"   "level.novaseq.rna"               
 [9] "novaseq.sub"                      "palette.novaseq"                 
[11] "palette.novaseq.renal_region_new" "palette.novaseq.rna"             
[13] "peak_mat"                         "processed_dir"

In [13]:
Idents(novaseq.sub) = novaseq.sub$renal_region_new
head(Idents(novaseq.sub))

R1.020,R2.020,R3.036,P1.01,B2 R1.067,R2.026,R3.067,P1.62,B2 
                       Cortex                       Papilla 
R1.061,R2.068,R3.015,P1.74,B3 R1.031,R2.058,R3.092,P1.84,B3 
                      Papilla                        Cortex 
R1.025,R2.004,R3.076,P1.61,B3 R1.067,R2.004,R3.014,P1.50,B2 
                      Medulla                       Papilla 
Levels: Cortex Medulla Papilla Renal Artery Ureter

### subset mpt from novaseq.sub because comparison is to be made bewtween two celltypes

In [14]:
mpt <- subset(novaseq.sub, renal_region_new %in% c(FindMarkers_ident_1, FindMarkers_ident_2))
mpt
table(mpt$celltype5_rna)

An object of class Seurat 
237522 features across 14841 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: SCT
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, umap.peaks, WNN.UMAP


      JGA       POD       PEC        PT PT_dediff  PT_VCAM1       tL1       tL2 
        0         0         0         0         0         0         0         0 
   tL-TAL      TAL1      TAL2      TAL3       DCT       CNT       PC1       PC2 
        0         0         0         0         0         0     11327      3514 
      ICA       ICB      Uro1      Uro2      ENDO      SMC1      SMC2      SMC3 
        0         0         0         0         0         0         0         0 
     Fib1      Fib2      Fib3        Ma        BT 
        0         0         0         0         0 

## load hli_446267_cells de_gene data frame

In [15]:
Sys.time()
options(future.globals.maxSize = 8000 * 1024^2)
load(de_gene_file)
Sys.time()

[1] "2023-07-03 14:59:33 CDT"

[1] "2023-07-03 14:59:33 CDT"

In [16]:
rownames(de_genes) = de_genes$names
de_genes_BK = de_genes
print(head(de_genes))
dim(de_genes)

          names           scores                 pvals             pvals_adj
SLC14A2 SLC14A2  31.957067489624 4.30935376039755e-224 8.02401670186024e-220
AUTS2     AUTS2  23.783052444458 5.00195195733028e-125 1.86272690890979e-121
PCDH7     PCDH7  21.848690032959 7.99948405213519e-106 2.48250655084595e-102
PDE3A     PDE3A 20.9934558868408   7.5269603772299e-98  2.15618464960032e-94
COL4A4   COL4A4 20.9076042175293  4.56596055585979e-97   1.2145455078587e-93
ANKS1B   ANKS1B 20.6729755401611  6.06586245403742e-95  1.50595145192236e-91
           logfoldchanges
SLC14A2  1.44827747344971
AUTS2    1.32176184654236
PCDH7    1.21994042396545
PDE3A   0.669078528881073
COL4A4   1.23974442481995
ANKS1B   1.49152863025665


[1] 37240     5

In [17]:
### keep DEGs with pvals_adj<0.05
p_val_cutoff = 0.05
peak_assay = "peaks"
de_genes <- de_genes[which(de_genes$pvals_adj < p_val_cutoff), ]
DefaultAssay(novaseq.sub) <- peak_assay
DefaultAssay(mpt) <- peak_assay

dim(de_genes)

[1] 684   5

In [18]:
Sys.time()
# gene_coords <- Signac:::CollapseToLongestTranscript(Annotation(novaseq.sub))
gene_coords <- Signac:::CollapseToLongestTranscript(Annotation(mpt))
Sys.time()

[1] "2023-07-03 14:59:33 CDT"

[1] "2023-07-03 14:59:39 CDT"

In [19]:
de_genes <- de_genes[which(rownames(de_genes) %in% gene_coords$gene_name), ]
dim(de_genes)
gene_list = rownames(de_genes)

[1] 573   5

## Next, make two de_gene sublists: gene_list_neg, gene_list_pos.

In [20]:
gene_list_neg = rownames(de_genes)[de_genes$logfoldchanges < 0]
print(length(gene_list_neg))
gene_list_pos = rownames(de_genes)[de_genes$logfoldchanges > 0]
print(length(gene_list_pos))

[1] 93
[1] 480


In [21]:
# de_genes[order(de_genes$logfoldchanges, decreasing = T),]

## run_peak_aen

In [22]:
Sys.time()

options(future.globals.maxSize = 8000 * 1024^2)

peak_results <- run_peak_aen(novaseq.sub, expr_mat, peak_mat, gene_list, lambda2 = 0.5, max_distance = 5e+05, num_bootstraps = 100)

aen_lists <- make_aen_lists(peak_results)

Sys.time()

[1] "2023-07-03 14:59:39 CDT"

Loading required package: gcdnet


Attaching package: ‘gcdnet’


The following object is masked from ‘package:stats4’:

    coef


The following objects are masked from ‘package:stats’:

    coef, predict


Loading required package: future


Attaching package: ‘future’


The following object is masked from ‘package:rtracklayer’:

    values


The following object is masked from ‘package:GenomicRanges’:

    values


The following object is masked from ‘package:IRanges’:

    values


The following object is masked from ‘package:S4Vectors’:

    values


Loading required package: future.apply



[1] "Omitted (mitochondrial) genes with no genomic peaks: USP9Y"
[1] "AEN completed in 11.07498087883"


[1] "2023-07-03 15:11:11 CDT"

## prepare_pseudocell_matrix for mpt

In [23]:
expr_mat_2 <- prepare_pseudocell_matrix(mpt, 
                                      assay = "SCT", 
                                      slot = "data", 
                                      cells_per_partition = 10, 
                                      find_neighbors = FALSE,
                                      reduction1 = "harmony_peaks",
                                      reduction2 = "harmony_RNA",
                                      dim_list = list(1:50, 1:50),
                                      k.nn = 5,
                                      seed = 489284)

Loading required package: Matrix


Attaching package: ‘Matrix’


The following object is masked from ‘package:S4Vectors’:

    expand


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: VISION

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDif

In [24]:
length(gene_list)
sum(gene_list %in% colnames(expr_mat_2))

[1] 573

[1] 573

### run_tf_aen

In [25]:
Sys.time()

tf_results <- run_tf_aen(mpt, expr_mat_2, peak_results, gene_list, lambda2 = 0.5)

regulator_tf_names <- unlist(novaseq.sub@assays$peaks@motifs@motif.names)

regulator_tf_names <- regulator_tf_names[which(regulator_tf_names %in% rownames(GetAssayData(novaseq.sub, assay = "SCT")))]

centrality_rankings <- rank_tfs_by_centrality(tf_results, novaseq.sub)

Sys.time()

[1] "2023-07-03 15:14:24 CDT"

[1] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: CDH18"
[1] "AEN completed in 5.80421022971471"


Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:future’:

    %->%, %<-%


The following objects are masked from ‘package:rtracklayer’:

    blocks, path


The following object is masked from ‘package:Biostrings’:

    union


The following object is masked from ‘package:XVector’:

    path


The following object is masked from ‘package:GenomicRanges’:

    union


The following object is masked from ‘package:IRanges’:

    union


The following object is masked from ‘package:S4Vectors’:

    union


The following objects are masked from ‘package:BiocGenerics’:

    normalize, path, union


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:purrr’:

    compose, simplify


The following object is masked from ‘package:tidyr’:

    crossing


The following object is masked from ‘package:tibble’:

    as_data_frame


The following objects a

[1] "2023-07-03 15:20:47 CDT"

In [26]:
print(Sys.time())
    
print("centrality_rankings - Betweenness:")
print(head(centrality_rankings$Betweenness,20))
print("centrality_rankings - PageRank:")
print(head(centrality_rankings$PageRank,20))

print(Sys.time())
print("calculating rank_tfs...")
tf_ranks = rank_tfs(tf_results, 
                    negative_genes = rownames(de_genes_BK)[which(de_genes_BK$logfoldchanges < 0)], 
                    seurat = mpt, 
                    num_cores =  1
    )
print(Sys.time())

[1] "2023-07-03 15:20:47 CDT"
[1] "centrality_rankings - Betweenness:"
 ESRRB   EMX1   SIM1   THRB   ATF3  GATA2   RORA   KLF7   NFIB    FOS ZNF423 
1312.5  274.5  248.5  116.0  106.0   87.0   67.5   67.0   55.5   21.0   20.0 
  RFX7   MBD2  GATA3   TCF4    EHF   ELF5   ETV6  ZFHX3   PAX2 
   3.0    2.0    1.5    0.0    0.0    0.0    0.0    0.0    0.0 
[1] "centrality_rankings - PageRank:"
      ESRRG        TCF4       MECOM        ETV6        ELF5        SOX5 
0.013222881 0.011148688 0.010483807 0.010420659 0.009861351 0.009631400 
       NFIA       FOXP2       RUNX1       ESRRB        LEF1        BCL6 
0.009626254 0.009270963 0.007843776 0.007680337 0.007038481 0.007032630 
      NR3C1       NR3C2       NFAT5       DNMT1       BACH2       RUNX2 
0.007024629 0.007021068 0.006757302 0.006724710 0.006527935 0.005734290 
      TEAD1      ARNTL2 
0.005325054 0.005286814 
[1] "2023-07-03 15:20:47 CDT"
[1] "calculating rank_tfs..."
[1] "2023-07-03 15:21:57 CDT"


In [27]:
head(tf_ranks,20)
tail(tf_ranks,20)

,TF_name,Score,SE
,<chr>,<dbl>,<dbl>
NFAT5,NFAT5,3.0041068,0.19495883
SOX5,SOX5,2.1615007,0.16968285
NFIA,NFIA,1.8481234,0.08766299
ETV6,ETV6,1.3505316,0.07739595
ELF5,ELF5,1.2992198,0.06255594
TCF4,TCF4,1.2959302,0.06054958
FOXP2,FOXP2,1.1881570,0.06500880
TEAD1,TEAD1,1.1138776,0.09578981
LCOR,LCOR,0.9313769,0.06950751


,TF_name,Score,SE
,<chr>,<dbl>,<dbl>
FOXO1,FOXO1,-0.1104399,0.02745432
HOXD10,HOXD10,-0.1468185,0.02892684
SIM1,SIM1,-0.1512984,0.03700012
MEF2A,MEF2A,-0.1543823,0.06492537
CREM,CREM,-0.1583656,0.02962007
NPAS3,NPAS3,-0.1631915,0.05219810
MITF,MITF,-0.1790865,0.04136360
GATA3,GATA3,-0.2026458,0.03237915
EMX1,EMX1,-0.2282337,0.02788013


In [28]:
save(list=c("gene_list", "aen_lists", "de_genes_BK", "tf_results", "tf_ranks", "regulator_tf_names", "centrality_rankings", "peak_results"), 
     file = file.path(processed_dir, paste0("RENIN_324701_cells_", FindMarkers_ident_1, "_vs_", FindMarkers_ident_2, ".RData")), compress = T)

In [29]:
Sys.time()

[1] "2023-07-03 15:22:10 CDT"